<a href="https://colab.research.google.com/github/M4RC3LL0/NLP/blob/main/Korpai_Joo_sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

eleres_ut = "../input/"
adatok = pd.read_csv(eleres_ut + "sudoku.csv")
try:
    adatok = pd.DataFrame({"feladvanyok": adatok["puzzle"], "megoldasok": adatok["solution"]})
except:
    pass
adatok.head()

adatok.info()
print("Feladvány:\n", np.array(list(map(int, list(adatok['feladvanyok'][0])))).reshape(9, 9))
print("Megoldás:\n", np.array(list(map(int, list(adatok['megoldasok'][0])))).reshape(9, 9))


class AdatGeneralas(Sequence):
    def __init__(self, df, batch_meret=32, alcsoport="tanitas", osszekever=False, info={}):
        super().__init__()
        self.df = df
        self.batch_meret = batch_meret
        self.osszekever = osszekever
        self.alcsoport = alcsoport
        self.info = info

        self.adat_utvonal = eleres_ut
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_meret))

    def on_epoch_end(self):
        self.indexek = np.arange(len(self.df))
        if self.osszekever:
            np.random.shuffle(self.indexek)

    def __getitem__(self, index):
        X = np.empty((self.batch_meret, 9, 9, 1))
        y = np.empty((self.batch_meret, 81, 1))
        indexek = self.indexek[index * self.batch_meret: (index + 1) * self.batch_meret]
        for i, f in enumerate(self.df['feladvanyok'].iloc[indexek]):
            self.info[index * self.batch_meret + i] = f
            X[i,] = (np.array(list(map(int, list(f)))).reshape((9, 9, 1)) / 9) - 0.5
        if self.alcsoport == 'tanitas':
            for i, f in enumerate(self.df['megoldasok'].iloc[indexek]):
                self.info[index * self.batch_meret + i] = f
                np.array(list(map(int, list(f))))
                y[i,] = np.array(list(map(int, list(f)))).reshape((81, 1)) - 1
        if self.alcsoport == 'tanitas':
            return X, y
        else:
            return X

modell = Sequential()

modell.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(9, 9, 1)))
modell.add(BatchNormalization())
modell.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
modell.add(BatchNormalization())
modell.add(Conv2D(128, kernel_size=(1, 1), activation='relu', padding='same'))

modell.add(Flatten())
modell.add(Dense(81 * 9))
modell.add(Reshape((-1, 9)))
modell.add(Activation('softmax'))

modell.compile(loss='sparse_categorical_crossentropy',  optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
modell.summary()

tanito_index = int(len(adatok) * 0.95)
adatok = adatok.sample(frac=1).reset_index(drop=True)
tanito_generalas = AdatGeneralas(adatok.iloc[:tanito_index], alcsoport="tanitas", batch_meret=640)
validacio_generalas = AdatGeneralas(adatok.iloc[tanito_index:], alcsoport="tanitas",  batch_meret=640)

tanito_generalas.__getitem__(4)[0].shape

from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
elmentesi_ut1 = "sulyok-javitas-{epoch:02d}-{val_accuracy:.2f}.hdf5"
elmentesi_ut2 = "legjobb_sulyok.hdf5"
ellenorzes1 = ModelCheckpoint(elmentesi_ut1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
ellenorzes2 = ModelCheckpoint(elmentesi_ut2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

csokkentes_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_lista = [ellenorzes1, ellenorzes2, csokkentes_lr]

elmenet = modell.fit_generator(tanito_generalas, validation_data = validacio_generalas, epochs = 5, verbose=1,callbacks=callbacks_lista )

modell.load_weights('legjobb_sulyok.hdf5')
def sudoku_megoldas_nn(model, feladvany):
    # Bemeneti Sudoku feladvány előfeldolgozása
    feladvany = feladvany.replace('\n', '').replace(' ', '')
    kezdo_tabla = np.array([int(j) for j in feladvany]).reshape((9, 9, 1))
    kezdo_tabla = (kezdo_tabla / 9) - 0.5

    while True:
        # Neurális hálózat használata a hiányzó cellák értékeinek előrejelzésére
        jovendobozok = model.predict(kezdo_tabla.reshape((1, 9, 9, 1))).squeeze()
        jovend = np.argmax(jovendobozok, axis=1).reshape((9, 9)) + 1
        valoszinuseg = np.around(np.max(jovendobozok, axis=1).reshape((9, 9)), 2)

        kezdo_tabla = ((kezdo_tabla + 0.5) * 9).reshape((9, 9))
        maszk = (kezdo_tabla == 0)

        if maszk.sum() == 0:
            # A Sudoku feladvány megoldva
            break

        valoszinuseg_uj = valoszinuseg * maszk

        ind = np.argmax(valoszinuseg_uj)
        x, y = (ind // 9), (ind % 9)

        ertek = jovend[x][y]
        kezdo_tabla[x][y] = ertek
        kezdo_tabla = (kezdo_tabla / 9) - 0.5

    megoldott_feladvany = ''.join(map(str, kezdo_tabla.flatten().astype(int)))
    megoldott_feladvany_halo = ""
    for i in range(9):
        for j in range(9):
            megoldott_feladvany_halo += megoldott_feladvany[i * 9 + j] + " "
            if j == 2 or j == 5:
                megoldott_feladvany_halo += "| "
        megoldott_feladvany_halo += "\n"
        if i == 2 or i == 5:
            megoldott_feladvany_halo += "-" * 21 + "\n"

    return megoldott_feladvany_halo

def sudoku_halo(feladvany):
    feladvany = feladvany.replace('\n', '').replace(' ', '')
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-"*21)

        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(feladvany[i*9 + j], end=" ")
        print()
uj_feladvany = '''
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
      '''

feladvany = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''

megoldott_feladvany_nn = sudoku_megoldas_nn(modell, feladvany)

# Kiírás a megoldott feladványra a hálózattal
print("Sudoku Megoldás (NN):")
sudoku_halo(megoldott_feladvany_nn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3625260 entries, 0 to 3625259
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   feladvanyok  object
 1   megoldasok   object
dtypes: object(2)
memory usage: 55.3+ MB
Feladvány:
 [[0 7 0 0 0 0 0 4 3]
 [0 4 0 0 0 9 6 1 0]
 [8 0 0 6 3 4 9 0 0]
 [0 9 4 0 5 2 0 0 0]
 [3 5 8 4 6 0 0 2 0]
 [0 0 0 8 0 0 5 3 0]
 [0 8 0 0 7 0 0 9 1]
 [9 0 2 1 0 0 0 0 5]
 [0 0 7 0 4 0 8 0 2]]
Megoldás:
 [[6 7 9 5 1 8 2 4 3]
 [5 4 3 7 2 9 6 1 8]
 [8 2 1 6 3 4 9 5 7]
 [7 9 4 3 5 2 1 8 6]
 [3 5 8 4 6 1 7 2 9]
 [2 1 6 8 9 7 5 3 4]
 [4 8 5 2 7 6 3 9 1]
 [9 6 2 1 8 3 4 7 5]
 [1 3 7 9 4 5 8 6 2]]
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 9, 9, 64)          640       
                                                                 
 batch_normalization_22 (Ba  (None, 9, 9, 64)          25

<ipython-input-14-26c20b247754>:98: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  elmenet = modell.fit_generator(tanito_generalas, validation_data = validacio_generalas, epochs = 5, verbose=1,callbacks=callbacks_lista )


3793/5382 [====================>.........] - ETA: 18:55 - loss: 0.5387 - accuracy: 0.1121

InvalidArgumentError: ignored